# SK ChatBot

Learning objectives:
- Difference between LLM and GPT models
- System vs user messages
- Working keeping a conversation
- Advantages of using SK:
  - HttpClient handling and built-in retry logic
  - Pre-defined models (Prompt & Completion)
  - Process prompts as pipes
  - Prompts and prompt templates (semantic functions)
  - Process native functions as semantic functions


## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.19.230804.2-preview"
#r "nuget: dotenv.net"

using System.Net.Http;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var depoymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {depoymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");

## Build a kernel instance

- Registration: `WithAzureChatCompletionServices` registers an ICompletion service with the kernel
- Retries: Kernel API calls come with default retry logic for timeouts and throttling.

In [ ]:
var httpclient = new HttpClient();
var kernel = new KernelBuilder()
    .WithAzureChatCompletionService(depoymentName,endpoint,apiKey,httpClient: httpclient)
    .Build();

## Get the registered service from the kernel and keep a conversation (a history)

In [ ]:
// Get an IChatCompletion service from the kernel
var chatGPT = kernel.GetService<IChatCompletion>();

// Keep a history
var systemMessage = "You are a general assistant. Say Hello, I'm your friendly bot. How can I assist you today?";
var conversation = (OpenAIChatHistory)chatGPT.CreateNewChat(systemMessage);

## Call the bot for the first time

In [ ]:
string assistantReply = await chatGPT.GenerateMessageAsync(conversation, new ChatRequestSettings());
Console.WriteLine($"\nBot: {assistantReply}");

conversation.AddAssistantMessage(assistantReply);

## Enter the while loop and process the user and system messages

In [ ]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");
    conversation.AddUserMessage(userMessage);
    if (userMessage == "history") {
        foreach (var message in conversation.Messages) {
            Console.WriteLine(message.Role+ ": " + message.Content);
        }
        continue;
    }
    string assistantReply = await chatGPT.GenerateMessageAsync(conversation, new ChatRequestSettings());
    // or
    //IReadOnlyList<ITextResult> completionResults = await chatGPT.GetCompletionsAsync(conversation, new CompleteRequestSettings() { MaxTokens = 100, Temperature = 0.4, TopP = 1 });
    //Console.WriteLine(await completionResults[0].GetCompletionAsync());
    //Console.WriteLine(completionResults[0].ModelResult.GetOpenAIChatResult().Usage.AsJson());
    //Console.WriteLine();

    Console.WriteLine($"\nBot: {assistantReply}");
    // Add the message to the conversation history
    conversation.AddAssistantMessage(assistantReply);
}

## Sample of using ChatCompletion as TextCompletion

In [ ]:
const string fnIdentifyHeadingsByRegionTemplate = @"For the following news headings, which news is out of {{$region}}: 
{{$news}}.";

var indentifyHeadingsSKFn = kernel.CreateSemanticFunction(fnIdentifyHeadingsByRegionTemplate, 
    maxTokens: 200, temperature: 0.5, topP: 1);

var news = @"Disaster in New Delhi
Train derailment kills many in Pakistan
US Team loses in Australia
Heat wave in the US";

var context = kernel.CreateNewContext();
context["region"] = "Asia";
context["news"] = news;

var bot_answer = await indentifyHeadingsSKFn.InvokeAsync(context);
Console.WriteLine(bot_answer);